# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to load
cities = "Resources/cities.csv"

# Read file
cities_df = pd.read_csv(cities)

# Preview dataframe
cities_df.head()

,Unnamed: 0,City,Country,City_ID,Date,Lat,Long,Cloudiness,Humidity,Max_Temp,Wind_Speed
0,0,Airai,TL,1651810,1584324058,-8.93,125.41,97,58,80.28,5.86
1,1,Kangaatsiaq,GL,3422683,1584324028,68.31,-53.46,100,92,0.99,12.53
2,2,Jamestown,US,5122534,1584324008,42.10,-79.24,1,39,30.99,4.70
3,3,Talcahuano,CL,3870282,1584324059,-36.72,-73.12,0,67,60.80,13.87
4,4,Slavyanka,RU,2016430,1584324059,42.86,131.38,81,41,34.86,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Obtain coordinates and humidity
coordinates = list(zip(cities_df["Lat"], cities_df["Long"]))
humidity = cities_df["Humidity"]

In [4]:
# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

#Add heatmap layer
heatmap_layer = gmaps.heatmap_layer(coordinates, weights = humidity)
fig.add_layer(heatmap_layer)

# Display map
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create a new data for weather criteria
weather_criteria = cities_df

# Drop any rows with null values
weather_criteria = weather_criteria.dropna()

# Define weather conditions
max_temp = (weather_criteria['Max_Temp'] <= 90) & (weather_criteria['Max_Temp'] > 60)
wind_speed = weather_criteria['Wind_Speed'] < 20
humidity = weather_criteria['Humidity'] < 30
cloudiness = weather_criteria['Cloudiness'] < 10

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# Narrow down the dataframe with the conditions above for the ideal list of cities
hotel_df = weather_criteria[max_temp & wind_speed & humidity & cloudiness].reset_index()

# Clean up dataframe
del hotel_df['Unnamed: 0']
del hotel_df['index']

# See narrowed down list
hotel_df

,City,Country,City_ID,Date,Lat,Long,Cloudiness,Humidity,Max_Temp,Wind_Speed
0,Yulara,AU,6355222,1584324013,-25.24,130.99,0,9,84.20,8.05
1,Jalu,LY,86049,1584324108,29.03,21.55,0,26,64.78,6.44
2,Bitkine,TD,2435124,1584324113,11.98,18.21,1,14,69.66,7.76
3,Atar,MR,2381334,1584323936,20.52,-13.05,0,26,63.57,7.65
4,Adrar,MR,2381972,1584324133,20.50,-10.07,0,20,67.33,12.62


In [12]:
# Add a "Hotel Name" column to the DataFrame
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,City_ID,Date,Lat,Long,Cloudiness,Humidity,Max_Temp,Wind_Speed,Hotel Name
0,Yulara,AU,6355222,1584324013,-25.24,130.99,0,9,84.20,8.05,
1,Jalu,LY,86049,1584324108,29.03,21.55,0,26,64.78,6.44,
2,Bitkine,TD,2435124,1584324113,11.98,18.21,1,14,69.66,7.76,
3,Atar,MR,2381334,1584323936,20.52,-13.05,0,26,63.57,7.65,
4,Adrar,MR,2381972,1584324133,20.50,-10.07,0,20,67.33,12.62,


In [16]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Hit the Google Places API for each city's coordinates.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Long"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Assemble url and make API request
    print(f"Calling results for index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response['results']
    
    # Store the first Hotel result into the DataFrame.
    try:
        print(f"First hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except:
        print("Missing result... skipping city.")
        
    print("------------")
    

Calling results for index 0: Yulara.
First hotel is Desert Gardens Hotel - Ayers Rock Resort.
------------
Calling results for index 1: Jalu.
First hotel is Jalu Hotel.
------------
Calling results for index 2: Bitkine.
Missing result... skipping city.
------------
Calling results for index 3: Atar.
First hotel is Oumou Elghoura.
------------
Calling results for index 4: Adrar.
Missing result... skipping city.
------------


In [17]:
# Show updated hotel df with added hotel names
hotel_df

,City,Country,City_ID,Date,Lat,Long,Cloudiness,Humidity,Max_Temp,Wind_Speed,Hotel Name
0,Yulara,AU,6355222,1584324013,-25.24,130.99,0,9,84.20,8.05,Desert Gardens Hotel - Ayers Rock Resort
1,Jalu,LY,86049,1584324108,29.03,21.55,0,26,64.78,6.44,Jalu Hotel
2,Bitkine,TD,2435124,1584324113,11.98,18.21,1,14,69.66,7.76,
3,Atar,MR,2381334,1584323936,20.52,-13.05,0,26,63.57,7.65,Oumou Elghoura
4,Adrar,MR,2381972,1584324133,20.50,-10.07,0,20,67.33,12.62,


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [19]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)

#Display map
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…